In [0]:
# !pip install googlemaps folium

In [0]:
chave = 'xxxxx'

In [0]:
# Databricks Notebook: VRP em Brasília (Asa Norte/Sul) com rotas reais no mapa
# Objetivo: (1) Traçar rotas reais (linhas) via Google Directions API
#           (2) Resolver VRP com metaheurística simples (Savings + 2-opt + SA) e exibir no mapa
#           (+) Calcular distância (km), custo de combustível (R$ 6,19) e emissões (kg CO2) por rota
# Requisitos: criar um Secret chamado GOOGLE_MAPS_API_KEY ou preencher manualmente a variável API_KEY

# =============================
# 0) Instalação de dependências
# =============================
# Em Databricks, rode esta célula uma vez
# %pip install googlemaps folium

# =============================
# 1) Imports e configuração
# =============================
import math
import random
import time
from typing import List, Tuple
import pandas as pd

try:
    import googlemaps
    import folium
    from folium.plugins import MarkerCluster
except Exception as e:
    raise RuntimeError("Instale as dependências com %pip install googlemaps folium")

# Use secret se preferir:
# API_KEY = dbutils.secrets.get(scope="meu_escopo", key="GOOGLE_MAPS_API_KEY")
API_KEY = chave  # <-- sua variável existente

gmaps = googlemaps.Client(key=API_KEY)

# =============================
# 2) Dados: depósito + agências (coords aproximadas)
# =============================
# Depósito: Sede I BB
deposito = {"id": 0, "nome": "Sede I BB", "lat": -15.784151683970695, "lng": -47.876520803590964, "demanda": 0}

# Amostra de agências/quadras – ajuste conforme necessidade
agencias = [
    {"id": 1, "nome": "Agência 504 Norte",  "lat": -15.776061525413336, "lng": -47.88760178301653, "demanda": 1},
    {"id": 2, "nome": "Agência 316 Norte",  "lat": -15.736449050328025, "lng": -47.896672541230295, "demanda": 1},
    {"id": 3, "nome": "Agência 214 Norte",  "lat": -15.742974605251662, "lng": -47.88671624356558, "demanda": 2},
    {"id": 4, "nome": "Agência PR",         "lat": -15.797450199472786, "lng": -47.87843220869015, "demanda": 1},
    {"id": 5, "nome": "Agência 203 Sul",    "lat": -15.809157077284526, "lng": -47.88875937163554, "demanda": 1},
    {"id": 6, "nome": "Agência 504 Sul",    "lat": -15.805235702140957, "lng": -47.89777060837944, "demanda": 2},
    {"id": 7, "nome": "Agência UnB",        "lat": -15.763046662434409, "lng": -47.87023123831819, "demanda": 2},
    {"id": 8, "nome": "Agência 516 Sul",    "lat": -15.827885706233321, "lng": -47.92817704258958, "demanda": 1},
]

VEICULOS = 3     # quantidade de veículos
CAPACIDADE = 3   # capacidade por veículo (soma das demandas)

# =============================
# 3) Google Maps: matrizes de duração (s) e distância (m) + direções
# =============================
Point = Tuple[float, float]
coords = [(deposito["lat"], deposito["lng"])] + [(a["lat"], a["lng"]) for a in agencias]
ids = [0] + [a["id"] for a in agencias]

def build_time_and_distance_matrices(coords: List[Point], mode: str = "driving"):
    """Retorna (duration_sec[n][n], distance_m[n][n])."""
    n = len(coords)
    dur = [[0]*n for _ in range(n)]
    dist = [[0]*n for _ in range(n)]
    result = gmaps.distance_matrix(origins=coords, destinations=coords, mode=mode)
    for i in range(n):
        for j in range(n):
            elem = result["rows"][i]["elements"][j]
            if elem.get("status") == "OK":
                dur[i][j] = elem["duration"]["value"]   # segundos
                dist[i][j] = elem["distance"]["value"]  # metros
            else:
                dur[i][j] = 10**9
                dist[i][j] = 10**9
    return dur, dist

def directions_polyline(src: Point, dst: Point, mode: str = "driving"):
    res = gmaps.directions(src, dst, mode=mode)
    if not res:
        return []
    poly = res[0].get("overview_polyline", {}).get("points")
    if not poly:
        return []
    return decode_polyline(poly)

def decode_polyline(encoded: str) -> List[Point]:
    points = []
    index = lat = lng = 0
    length = len(encoded)
    while index < length:
        shift = result = 0
        while True:
            b = ord(encoded[index]) - 63
            index += 1
            result |= (b & 0x1f) << shift
            shift += 5
            if b < 0x20:
                break
        dlat = ~(result >> 1) if (result & 1) else (result >> 1)
        lat += dlat
        shift = result = 0
        while True:
            b = ord(encoded[index]) - 63
            index += 1
            result |= (b & 0x1f) << shift
            shift += 5
            if b < 0x20:
                break
        dlng = ~(result >> 1) if (result & 1) else (result >> 1)
        lng += dlng
        points.append((lat / 1e5, lng / 1e5))
    return points

# =============================
# 4) Matrizes (tempo e distância)
# =============================
print("Consultando Google Distance Matrix...")
duration_matrix, distance_matrix = build_time_and_distance_matrices(coords)

# =============================
# 5) Metaheurística VRP: Savings + 2-opt + SA
# =============================
def route_time_sec(route: List[int]) -> int:
    """Soma as durações (s) ao longo da rota."""
    cost = 0
    for i in range(len(route)-1):
        a = ids.index(route[i])
        b = ids.index(route[i+1])
        cost += duration_matrix[a][b]
    return cost

def route_dist_km(route: List[int]) -> float:
    """Soma as distâncias (km) ao longo da rota."""
    d_m = 0
    for i in range(len(route)-1):
        a = ids.index(route[i])
        b = ids.index(route[i+1])
        d_m += distance_matrix[a][b]
    return d_m / 1000.0

def total_demand(route: List[int]) -> int:
    if not route: return 0
    return sum(a["demanda"] for a in agencias if a["id"] in route)

id_to_ag = {a["id"]: a for a in agencias}
def get_demand(node_id: int) -> int:
    return 0 if node_id == 0 else id_to_ag[node_id]["demanda"]

# Rotas iniciais (cada cliente sozinho)
initial_routes = [[0, a["id"], 0] for a in agencias]

# Savings
savings = []
for i in range(1, len(ids)):
    for j in range(1, len(ids)):
        if i == j: continue
        ci0 = duration_matrix[i][0]
        c0j = duration_matrix[0][j]
        cij = duration_matrix[i][j]
        savings.append(((ids[i], ids[j]), ci0 + c0j - cij))
savings.sort(key=lambda x: x[1], reverse=True)

def find_route_containing(routes, node):
    for r_idx, r in enumerate(routes):
        if node in r[1:-1]:
            return r_idx
    return -1

def is_route_end(route, node):
    return (route[1] == node) or (route[-2] == node)

routes = initial_routes[:]
for (i, j), s in savings:
    ri = find_route_containing(routes, i)
    rj = find_route_containing(routes, j)
    if ri == -1 or rj == -1 or ri == rj:
        continue
    r_i = routes[ri]
    r_j = routes[rj]
    if not is_route_end(r_i, i) or not is_route_end(r_j, j):
        continue
    if r_i[1] != i:
        r_i = [0] + list(reversed(r_i[1:-1])) + [0]
    if r_j[-2] != j:
        r_j = [0] + list(reversed(r_j[1:-1])) + [0]
    merged = r_i[:-1] + r_j[1:]
    if total_demand(merged) <= CAPACIDADE:
        routes[ri] = merged
        routes.pop(rj)
    if len(routes) <= VEICULOS:
        break

while len(routes) > VEICULOS:
    routes.sort(key=lambda r: total_demand(r))
    r1 = routes.pop(0)
    merged = None
    for k in range(len(routes)):
        cand = routes[k]
        for variant in [r1, [0]+list(reversed(r1[1:-1]))+[0]]:
            for variant2 in [cand, [0]+list(reversed(cand[1:-1]))+[0]]:
                trial = variant[:-1] + variant2[1:]
                if total_demand(trial) <= CAPACIDADE:
                    merged = (k, trial); break
            if merged: break
        if merged:
            idx, tr = merged
            routes[idx] = tr
            break
    if not merged:
        routes.append(r1)
        break

# 2-opt
def two_opt(route, max_iter: int = 200):
    best = route[:]
    best_cost = route_time_sec(best)
    improved, it = True, 0
    while improved and it < max_iter:
        improved, it = False, it+1
        for i in range(1, len(best)-2):
            for j in range(i+1, len(best)-1):
                if j - i == 1: continue
                new_route = best[:]
                new_route[i:j] = reversed(new_route[i:j])
                c = route_time_sec(new_route)
                if c < best_cost:
                    best, best_cost = new_route, c
                    improved = True
    return best

routes = [two_opt(r) for r in routes]

# Simulated Annealing (troca entre rotas)
def total_time_all(routes): return sum(route_time_sec(r) for r in routes)

def neighbors_swap_between(routes):
    import copy
    new_routes = copy.deepcopy(routes)
    if len(new_routes) < 2: return new_routes
    r1, r2 = random.sample(range(len(new_routes)), 2)
    cand1 = [n for n in new_routes[r1][1:-1]]
    cand2 = [n for n in new_routes[r2][1:-1]]
    if not cand1 or not cand2: return new_routes
    a = random.choice(cand1); b = random.choice(cand2)
    i1 = new_routes[r1].index(a); i2 = new_routes[r2].index(b)
    new_routes[r1][i1], new_routes[r2][i2] = new_routes[r2][i2], new_routes[r1][i1]
    if total_demand(new_routes[r1]) > CAPACIDADE or total_demand(new_routes[r2]) > CAPACIDADE:
        return routes
    return new_routes

T, alpha, iters = 1.0, 0.995, 400
best, best_cost = routes[:], total_time_all(routes)
cur, cur_cost = routes[:], best_cost
for _ in range(iters):
    nxt = neighbors_swap_between(cur)
    nxt_cost = total_time_all(nxt)
    if nxt_cost < cur_cost or random.random() < math.exp((cur_cost - nxt_cost)/max(T,1e-9)):
        cur, cur_cost = nxt, nxt_cost
        if cur_cost < best_cost:
            best, best_cost = cur, cur_cost
    T *= alpha
routes = best

print("Rotas finais (IDs):")
for r in routes:
    print(r)
print("Custo total (segundos):", best_cost)

# =============================
# 6) Mapa folium + tabela com tempos e custos/CO2
# =============================
def popup_txt(node_id: int) -> str:
    if node_id == 0: return deposito["nome"]
    a = id_to_ag[node_id]
    return f"{a['nome']} (demanda: {a['demanda']})"

m = folium.Map(location=[deposito["lat"], deposito["lng"]], zoom_start=13)
mc = MarkerCluster().add_to(m)

folium.Marker(
    [deposito["lat"], deposito["lng"]],
    popup=deposito["nome"],
    icon=folium.Icon(color="red", icon="home")
).add_to(mc)

for a in agencias:
    folium.Marker(
        [a["lat"], a["lng"]],
        popup=f"{a['nome']} | demanda={a['demanda']}",
        icon=folium.Icon(color="blue", icon="building")
    ).add_to(mc)

route_colors = ["blue", "green", "purple", "orange", "cadetblue", "darkred"]

for idx, r in enumerate(routes):
    color = route_colors[idx % len(route_colors)]
    for u, v in zip(r[:-1], r[1:]):
        src = coords[ids.index(u)]
        dst = coords[ids.index(v)]
        pts = directions_polyline(src, dst)
        if not pts: pts = [src, dst]
        folium.PolyLine(pts, weight=5, opacity=0.7, color=color).add_to(m)

# ===== Novas métricas: distância (km), combustível, custo, CO2 =====
# Parâmetros operacionais
preco_combustivel = 6.19   # R$/litro
consumo_km_por_litro = 12  # km/L (carro)
emissao_kg_por_km = 0.2    # kg CO2 por km (estimativa média)

rows = []
for r in routes:
    tempo_min = round(route_time_sec(r) / 60.0, 1)
    dist_km = round(route_dist_km(r), 2)
    combustivel_litros = round(dist_km / consumo_km_por_litro, 3)
    custo_combustivel = round(combustivel_litros * preco_combustivel, 2)
    emissoes_kg = round(dist_km * emissao_kg_por_km, 2)
    rows.append({
        "rota": " -> ".join(popup_txt(n) for n in r),
        "tempo_estimado_min": tempo_min,
        "dist_km": dist_km,
        "combustivel_litros": combustivel_litros,
        "custo_combustivel": custo_combustivel,
        "emissoes_kgCO2": emissoes_kg
    })

df_rotas = pd.DataFrame(rows)

# Totais agregados (úteis para o slide de KPIs)
totais = {
    "total_tempo_min": round(df_rotas["tempo_estimado_min"].sum(), 1),
    "total_distancia_km": round(df_rotas["dist_km"].sum(), 2),
    "total_combustivel_l": round(df_rotas["combustivel_litros"].sum(), 2),
    "total_custo_combustivel_R$": round(df_rotas["custo_combustivel"].sum(), 2),
    "total_emissoes_kgCO2": round(df_rotas["emissoes_kgCO2"].sum(), 2),
}

# Exibir tabela e totais
try:
    display(df_rotas)
    display(pd.DataFrame([totais]))
except Exception:
    print(df_rotas)
    print(totais)


Consultando Google Distance Matrix...
Rotas finais (IDs):
[0, 1, 7, 0]
[0, 3, 2, 0]
[0, 8, 6, 0]
[0, 4, 5, 0]
Custo total (segundos): 6727


rota,tempo_estimado_min,dist_km,combustivel_litros,custo_combustivel,emissoes_kgCO2
Sede I BB -> Agência 504 Norte (demanda: 1) -> Agência UnB (demanda: 2) -> Sede I BB,24.7,11.41,0.951,5.89,2.28
Sede I BB -> Agência 214 Norte (demanda: 2) -> Agência 316 Norte (demanda: 1) -> Sede I BB,26.9,18.69,1.558,9.64,3.74
Sede I BB -> Agência 516 Sul (demanda: 1) -> Agência 504 Sul (demanda: 2) -> Sede I BB,37.5,19.32,1.61,9.97,3.86
Sede I BB -> Agência PR (demanda: 1) -> Agência 203 Sul (demanda: 1) -> Sede I BB,23.1,12.27,1.022,6.33,2.45


total_tempo_min,total_distancia_km,total_combustivel_l,total_custo_combustivel_R$,total_emissoes_kgCO2
112.2,61.69,5.14,31.83,12.33


In [0]:
# Exibir o mapa interativo no Databricks
try:
    html = m._repr_html_()
    displayHTML(html)
except Exception:
    m.save("vrp_brasilia.html")
    print("Mapa salvo: vrp_brasilia.html")

Make this Notebook Trusted to load map: File -> Trust Notebook <iframe srcdoc="<!DOCTYPE html>
<html>
<head>
 
 <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
 <script src="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.js"></script>
 <script src="https://code.jquery.com/jquery-3.7.1.min.js"></script>
 <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/js/bootstrap.bundle.min.js"></script>
 <script src="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js"></script>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.9.3/dist/leaflet.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.2/dist/css/bootstrap.min.css"/>
 <link rel="stylesheet" href="https://netdna.bootstrapcdn.com/bootstrap/3.0.0/css/bootstrap-glyphicons.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/@fortawesome/fontawesome-free@6.2.0/css/all.min.css"/>
 <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css"/>
 <link rel="stylesheet" href="https://cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css"/>
 
 <meta name="viewport" content="width=device-width,
 initial-scale=1.0, maximum-scale=1.0, user-scalable=no" />
 <style>
 #map_2e6b04dfbdd0b338ecc24dcfa9080e2b {
 position: relative;
 width: 100.0%;
 height: 100.0%;
 left: 0.0%;
 top: 0.0%;
 }
 .leaflet-container { font-size: 1rem; }
 </style>

 <style>html, body {
 width: 100%;
 height: 100%;
 margin: 0;
 padding: 0;
 }
 </style>

 <style>#map {
 position:absolute;
 top:0;
 bottom:0;
 right:0;
 left:0;
 }
 </style>

 <script>
 L_NO_TOUCH = false;
 L_DISABLE_3D = false;
 </script>

 
 <script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet.markercluster/1.1.0/leaflet.markercluster.js"></script>
 <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/leaflet.markercluster/1.1.0/MarkerCluster.css"/>
 <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/leaflet.markercluster/1.1.0/MarkerCluster.Default.css"/>
</head>
<body>
 
 
 <div class="folium-map" id="map_2e6b04dfbdd0b338ecc24dcfa9080e2b" ></div>
 
</body>
<script>
 
 
 var map_2e6b04dfbdd0b338ecc24dcfa9080e2b = L.map(
 "map_2e6b04dfbdd0b338ecc24dcfa9080e2b",
 {
 center: [-15.784151683970695, -47.876520803590964],
 crs: L.CRS.EPSG3857,
 ...{
 "zoom": 13,
 "zoomControl": true,
 "preferCanvas": false,
}

 }
 );

 

 
 
 var tile_layer_1ced5d19aff465d3a90e2d836f95ac42 = L.tileLayer(
 "https://tile.openstreetmap.org/{z}/{x}/{y}.png",
 {
 "minZoom": 0,
 "maxZoom": 19,
 "maxNativeZoom": 19,
 "noWrap": false,
 "attribution": "\u0026copy; \u003ca href=\"https://www.openstreetmap.org/copyright\"\u003eOpenStreetMap\u003c/a\u003e contributors",
 "subdomains": "abc",
 "detectRetina": false,
 "tms": false,
 "opacity": 1,
}

 );
 
 
 tile_layer_1ced5d19aff465d3a90e2d836f95ac42.addTo(map_2e6b04dfbdd0b338ecc24dcfa9080e2b);
 
 
 var marker_cluster_6e8f41350fa4e71e2ff9d80ff7bd8792 = L.markerClusterGroup(
 {
}
 );
 
 
 var marker_3d908bbaf548628325976d59c1094a89 = L.marker(
 [-15.784151683970695, -47.876520803590964],
 {
}
 ).addTo(marker_cluster_6e8f41350fa4e71e2ff9d80ff7bd8792);
 
 
 var icon_9009d704140561ffe95ed05488b412cb = L.AwesomeMarkers.icon(
 {
 "markerColor": "red",
 "iconColor": "white",
 "icon": "home",
 "prefix": "glyphicon",
 "extraClasses": "fa-rotate-0",
}
 );
 
 
 var popup_da9e0cf0f05da5f489e3775fed7792b3 = L.popup({
 "maxWidth": "100%",
});

 
 
 var html_6d99aeea242fc2e780f0abc882d50899 = $(`<div id="html_6d99aeea242fc2e780f0abc882d50899" style="width: 100.0%; height: 100.0%;">Sede I BB</div>`)[0];
 popup_da9e0cf0f05da5f489e3775fed7792b3.setContent(html_6d99aeea242fc2e780f0abc882d50899);
 
 

 marker_3d908bbaf548628325976d59c1094a89.bindPopup(popup_da9e0cf0f05da5f489e3775fed7792b3)
 ;

 
 
 
 marker_3d908bbaf548628325976d59c1094a89.setIcon(icon_9009d704140561ffe95ed0